In [ ]:
!pip install virtualenv
!mkdir my_virtualenv
%cd my_virtualenv
!virtualenv my_env
!source my_env/bin/activate

In [ ]:
!pip install -qU cohere
!pip install -qU langchain
!pip install -qU pypdf
!pip install -qU tiktoken
!pip install -qU chromadb

In [ ]:
! pip install weasyprint
! pip install pdfkit
! pip install wkhtmltopdf

In [4]:
from getpass import getpass
cohere_key = getpass()

··········


In [ ]:
import cohere
co = cohere.Client(cohere_key)
response = co.generate(
  model='command-nightly',
  prompt='write a description about Cohere',
  max_tokens=300,
  temperature=0.9,
  k=0,
  p=0.75,
  stop_sequences=[],
  return_likelihoods='NONE')
print('Prediction: {}'.format(response.generations[0].text))

In [11]:
response.generations[0].text

"Cohere is a company that specializes in bringing the power of large language models to businesses. Cohere's cutting-edge technology empowers businesses to create tailored, data-driven solutions that enable them to optimize operations, automate customer interactions, and enhance content generation, all while delivering exceptional user experiences. \n\nThe company's innovative approach to Natural Language Processing (NLP) has allowed businesses to break through the limitations of traditional language models, unlocking new capabilities and driving digital transformation across a range of industries. With a focus on helping businesses to future-proof their operations, Cohere's advanced language models and NLP tools are revolutionizing the way companies operate, providing an efficient and tailored experience. \n\nCohere's mission is to empower every business to leverage the latest and most effective language AI. The company aims to do this by providing an accessible, reliable, and robust 

In [ ]:
from langchain.embeddings import CohereEmbeddings
embeddings = CohereEmbeddings(cohere_api_key=cohere_key)
text = "This is a test document."
query_result = embeddings.embed_query(text)
doc_result = embeddings.embed_documents([text])
print(doc_result)

In [14]:
len(query_result)

4096

In [15]:
import numpy as np
np.array(doc_result).shape

(1, 4096)

In [16]:
import weasyprint
import os

In [17]:
def create_pdf(url):
  pdf = weasyprint.HTML(url).write_pdf()
  output_dir="pdf_dir"
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)
  file_path = os.path.join(output_dir,'sample.pdf')
  with open(file_path,'wb') as f:
    f.write(pdf)

In [ ]:
url = "https://medium.com/@mygreatlearning/50-nlp-interview-questions-and-answers-with-explanations-6ea4af32cbda"
create_pdf(url)

In [20]:
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader,PyPDFDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import VectorDBQA
from langchain.llms import Cohere
import os
directory = "/content/my_virtualenv/pdf_dir"
loader = PyPDFDirectoryLoader(directory)
#loader = PyPDFLoader("/content/drive/MyDrive/ChatGPT/Shared/ChromaDB/Data/Australia - Wikipedia2.pdf")
pages = loader.load_and_split()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs = text_splitter.split_documents(pages)

os.environ["COHERE_API_KEY"] = cohere_key

In [21]:
embeddings = CohereEmbeddings()
db = Chroma.from_documents(docs, embeddings)

In [24]:
docs[5]

Document(page_content='13. T okens are converted into numbers before giving to\nany Neural Networka. True\nb. False\nAns:  a)\nIn NLP, all words are converted into a number before feeding to a Neural Network.\n14. identify the odd one outa. nltk\nb. scikit learn\nc. SpaCy\nd. BERT\nAns:  d)\nAll the ones mentioned are NLP libraries except BERT, which is a word embedding\n15. TF-IDF helps you to establish?a. most frequently occurring word in the document\nb. most important word in the document\nAns:  b)\nTF-IDF helps to establish how important a particular word is in the context of the\ndocument corpus. TF-IDF takes into account the number of times the word appears\nin the document and offset by the number of documents that appear in the corpus.\nTF is the frequency of term divided by a total number of terms in the document.\nIDF is obtained by dividing the total number of documents by the number of\ndocuments containing the term and then taking the logarithm of that quotient.\nTf.idf i

In [25]:
db

In [26]:
qa = VectorDBQA.from_chain_type(llm=Cohere(), chain_type="stuff", vectorstore=db)
query = "Tell me about NLP."
qa.run(query)

/usr/local/lib/python3.10/dist-packages/langchain/chains/retrieval_qa/base.py:256: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' Natural Language Processing (NLP) is a branch of artificial intelligence that focuses on the interaction between computers and human language. Its objective is to enable machines to comprehend, analyze, and interpret human language with the purpose of producing actionable insights. NLP combines computational linguistics, a branch of linguistics that deals with language and their syntax as well as semantics, with statistical, machine, and deep learning to enable intuitive communication between humans and machines. \n\nNLP is important because it allows machines to understand and process human language in order to perform tasks such as text translation, sentiment analysis, and text summarization. These tasks require the machine to understand the context, semantics, and syntax of the language in order to provide an accurate response.\n\nNLP is widely used in various industries such as healthcare, finance, and customer support to name a few. Its application is vast, ranging from language

In [27]:
query = "Describe about stemming and lemmatize."
qa.invoke(query)

{'query': 'Describe about stemming and lemmatize.',
 'result': " Stemming and lemmatization are two different processes for reducing the words in a language to their root, which is the core part of a word that holds its primary meaning. However, they differ in how they achieve this. Stemming is a more straightforward process that removes prefixes and suffixes from words to reach the root, while lemmatization uses the language's grammar and vocabulary to reduce words to their root, often by changing the word according to the rules of the language. Lemmatization usually produces longer roots than stemming, as it tries to maintain the word's original meaning and grammatical properties, while stemming may produce roots that aren't actual words. \n\nBoth are often used in natural language processing to help computers understand and process large amounts of text, such as in text classification, information retrieval, and word similarity calculations. "}